In [65]:
import pandas as pd
import numpy as np
import seaborn as sns
import lightgbm as lgbm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.base import clone

In [75]:
dataset = pd.read_csv('result_3.csv', index_col='client_id').drop('Unnamed: 0', axis=1)

In [14]:
dataset.head()

,age,gender,stores_count,mode_store_id,transaction_id,sum_regular_points_received,sum_express_points_received,sum_regular_points_spent,sum_express_points_spent,mean_regular_points_received,...,sum_last_30_days_product_quantity_feat,mean_last_30_days_product_quantity_feat,max_last_30_days_product_quantity_feat,min_last_30_days_product_quantity_feat,mean_product_quantity_store,diff_mean_product_quantity_total_store,diff_mean_product_quantity_last30_store,mean_sums_per_store,diff_purchase_sums_total_store,diff_purchase_sums_last30_store
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,45,U,3,017c89b915,4,392.7,0.0,0.0,0.0,7.551923,...,19.0,9.500000,13.0,6.0,5.196581,8.303419,4.303419,277.829145,422.920855,333.170855
000036f903,72,F,5,6381a55c22,32,348.3,360.0,0.0,0.0,2.150000,...,54.0,6.000000,11.0,2.0,6.235110,-0.953860,-0.235110,429.603072,-123.196822,-99.825294
000048b7a6,68,F,2,1a01e2a7d8,8,218.8,0.0,0.0,0.0,3.907143,...,8.0,4.000000,5.0,3.0,8.475936,2.899064,-4.475936,451.454465,20.045535,-102.454465
000073194a,60,F,1,9e8fcd116d,17,518.8,0.0,-576.0,0.0,6.326829,...,46.0,6.571429,17.0,1.0,11.588968,-4.118380,-5.017539,748.605569,-183.817334,-145.266998
00007c7133,67,U,2,01ec6c98d7,11,459.7,0.0,-1680.0,0.0,5.538554,...,6.0,6.000000,6.0,6.0,6.231788,1.313666,-0.231788,344.526998,266.367548,35.473002


In [23]:
df_train = pd.read_csv('uplift_train.csv', index_col='client_id')
df_test = pd.read_csv('uplift_test.csv', index_col='client_id')
df_clients = pd.read_csv('clients.csv', index_col='client_id', parse_dates=['first_issue_date', 'first_redeem_date'])

In [24]:
#добавить фичи с baseline

In [76]:
dataset['first_issue_unixtime'] = df_clients['first_issue_date'].values.astype(int)/10**9
dataset['first_redeem_unixtime'] = df_clients['first_redeem_date'].values.astype(int)/10**9
dataset['first_redeem_delay'] = dataset['first_redeem_unixtime'] - dataset['first_issue_unixtime']
dataset['gender_M'] = (dataset['gender'] == 'M').astype(int)
dataset['gender_F'] = (dataset['gender'] == 'F').astype(int)
dataset['gender_U'] = (dataset['gender'] == 'U').astype(int)

,age,gender,stores_count,mode_store_id,transaction_id,sum_regular_points_received,sum_express_points_received,sum_regular_points_spent,sum_express_points_spent,mean_regular_points_received,...,diff_mean_product_quantity_last30_store,mean_sums_per_store,diff_purchase_sums_total_store,diff_purchase_sums_last30_store,first_issue_unixtime,first_redeem_unixtime,first_redeem_delay,gender_M,gender_F,gender_U
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,45,U,3,017c89b915,4,392.7,0.0,0.0,0.0,7.551923,...,4.303419,277.829145,422.920855,333.170855,-1.639580,1.769781,3.409360,0,0,1
000036f903,72,F,5,6381a55c22,32,348.3,360.0,0.0,0.0,2.150000,...,-0.235110,429.603072,-123.196822,-99.825294,-1.786588,-0.133929,1.652659,0,1,0
000048b7a6,68,F,2,1a01e2a7d8,8,218.8,0.0,0.0,0.0,3.907143,...,-4.475936,451.454465,20.045535,-102.454465,-0.270457,0.000000,0.270457,0,1,0
000073194a,60,F,1,9e8fcd116d,17,518.8,0.0,-576.0,0.0,6.326829,...,-5.017539,748.605569,-183.817334,-145.266998,-1.523791,-1.793599,-0.269808,0,1,0
00007c7133,67,U,2,01ec6c98d7,11,459.7,0.0,-1680.0,0.0,5.538554,...,-0.231788,344.526998,266.367548,35.473002,-1.639897,1.653310,3.293207,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffece623e,67,U,3,75156f2a80,24,287.8,0.0,-544.0,0.0,2.459829,...,-2.223408,314.573589,-42.489839,-131.009589,-0.596172,0.961591,1.557763,0,0,1
ffff3dfff8,56,F,1,3465db4011,17,1040.7,0.0,-728.0,0.0,10.840625,...,2.025641,414.295112,13.048417,65.594888,-0.457501,-1.978659,-1.521159,0,1,0
ffffaab9da,23,F,1,46972d1c44,7,394.9,0.0,-66.0,0.0,6.581667,...,11.017218,291.860324,249.711105,266.139676,-0.588319,-0.832515,-0.244196,0,1,0


In [34]:
df_clients['first_issue_date'].values.astype(int)

array([-1639579648, -1786587648,  -270457344, ...,  -588318720,
        1081878528, -2125426688])

In [47]:
dataset.isnull().sum()[dataset.isnull().sum() != 0]

std_regular_points_received                    763
std_express_points_received                    763
std_regular_points_spent                       763
std_express_points_spent                       763
std_by_transaction_regular_points_received    7707
std_by_transaction_express_points_received    7707
std_by_transaction_regular_points_spent       7707
std_by_transaction_express_points_spent       7707
std_product_quantity                          7707
std_trn_sum_from_iss                          7707
std_purchase_sums                             7707
mean_product_quantity_store                   4009
diff_mean_product_quantity_total_store        4009
diff_mean_product_quantity_last30_store       4009
mean_sums_per_store                           4009
diff_purchase_sums_total_store                4009
diff_purchase_sums_last30_store               4009
dtype: int64

In [48]:
some = pd.read_csv('purchases.csv')

In [ ]:
tmp = some.groupbyby(['client_id', 'transaction_id'])

In [56]:
dataset.mode_store_id.isnull().sum()

0

In [55]:
dataset.fillna(0) # Другой способ заполнения

,age,gender,stores_count,mode_store_id,transaction_id,sum_regular_points_received,sum_express_points_received,sum_regular_points_spent,sum_express_points_spent,mean_regular_points_received,...,diff_mean_product_quantity_last30_store,mean_sums_per_store,diff_purchase_sums_total_store,diff_purchase_sums_last30_store,first_issue_unixtime,first_redeem_unixtime,first_redeem_delay,gender_M,gender_F,gender_U
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,45,U,3,017c89b915,4,392.7,0.0,0.0,0.0,7.551923,...,4.303419,277.829145,422.920855,333.170855,-1.639580,1.769781,3.409360,0,0,1
000036f903,72,F,5,6381a55c22,32,348.3,360.0,0.0,0.0,2.150000,...,-0.235110,429.603072,-123.196822,-99.825294,-1.786588,-0.133929,1.652659,0,1,0
000048b7a6,68,F,2,1a01e2a7d8,8,218.8,0.0,0.0,0.0,3.907143,...,-4.475936,451.454465,20.045535,-102.454465,-0.270457,0.000000,0.270457,0,1,0
000073194a,60,F,1,9e8fcd116d,17,518.8,0.0,-576.0,0.0,6.326829,...,-5.017539,748.605569,-183.817334,-145.266998,-1.523791,-1.793599,-0.269808,0,1,0
00007c7133,67,U,2,01ec6c98d7,11,459.7,0.0,-1680.0,0.0,5.538554,...,-0.231788,344.526998,266.367548,35.473002,-1.639897,1.653310,3.293207,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffece623e,67,U,3,75156f2a80,24,287.8,0.0,-544.0,0.0,2.459829,...,-2.223408,314.573589,-42.489839,-131.009589,-0.596172,0.961591,1.557763,0,0,1
ffff3dfff8,56,F,1,3465db4011,17,1040.7,0.0,-728.0,0.0,10.840625,...,2.025641,414.295112,13.048417,65.594888,-0.457501,-1.978659,-1.521159,0,1,0
ffffaab9da,23,F,1,46972d1c44,7,394.9,0.0,-66.0,0.0,6.581667,...,11.017218,291.860324,249.711105,266.139676,-0.588319,-0.832515,-0.244196,0,1,0


In [57]:
indices_train = df_train.index
indices_test = df_test.index
indices_learn, indices_valid = train_test_split(df_train.index, test_size=0.3, random_state=123)

In [59]:
def uplift_fit_predict(model, X_train, treatment_train, target_train, X_test):
    X_treatment, y_treatment = X_train[treatment_train == 1, :], target_train[treatment_train == 1]
    X_control, y_control = X_train[treatment_train == 0, :], target_train[treatment_train == 0]
    model_treatment = clone(model).fit(X_treatment, y_treatment)
    model_control = clone(model).fit(X_control, y_control)
    predict_treatment = model_treatment.predict_proba(X_test)[:, 1]
    predict_control = model_control.predict_proba(X_test)[:, 1]
    predict_uplift = predict_treatment - predict_control
    return predict_uplift


def uplift_score(prediction, treatment, target, rate=0.3):
    order = np.argsort(-prediction)
    treatment_n = int((treatment == 1).sum() * rate)
    treatment_p = target[order][treatment[order] == 1][:treatment_n].mean()
    control_n = int((treatment == 0).sum() * rate)
    control_p = target[order][treatment[order] == 0][:control_n].mean()
    score = treatment_p - control_p
    return score


In [78]:
dataset.drop(['gender'], axis=1, inplace=True)

In [80]:
dataset.to_csv('result_4.csv')

In [73]:
valid_uplift = uplift_fit_predict(
    model=GradientBoostingClassifier(**params),
    X_train=dataset.loc[indices_learn, :].fillna(0).values,
    treatment_train=df_train.loc[indices_learn, 'treatment_flg'].values,
    target_train=df_train.loc[indices_learn, 'target'].values,
    X_test=dataset.loc[indices_valid, :].fillna(0).values,
)
valid_score = uplift_score(
    valid_uplift,
    treatment=df_train.loc[indices_valid, 'treatment_flg'].values,
    target=df_train.loc[indices_valid, 'target'].values,
)
print('Validation score:', valid_score)

      Iter       Train Loss   Remaining Time 
         1           1.2995            5.15m
         2           1.2888            5.13m
         3           1.2786            5.11m
         4           1.2691            5.09m
         5           1.2601            5.09m
         6           1.2517            5.07m
         7           1.2437            5.06m
         8           1.2361            5.05m
         9           1.2290            5.04m
        10           1.2222            5.03m
        20           1.1713            4.91m
        30           1.1407            4.80m
        40           1.1216            4.70m
        50           1.1094            4.61m
        60           1.1014            4.51m
        70           1.0958            4.41m
        80           1.0918            4.30m
        90           1.0888            4.19m
       100           1.0861            4.09m
       200           1.0709            3.04m
      Iter       Train Loss   Remaining Time 
        

In [72]:
params = {
    'learning_rate': 0.03,
    'verbose': 1,
    'n_iter_no_change': 10,
    'max_depth': 4,
    'n_estimators': 500,
    'random_state': 174
}

In [ ]:
import lightgbm as lgbm
params = {'learning_rate':0.03,'max_depth':4,'num_leaves':20,
             'min_data_in_leaf':3, 'application':'binary',
             'subsample':0.8, 'colsample_bytree': 0.8,
             'reg_alpha':0.01,'data_random_seed':42,'metric':'binary_logloss',
             'max_bin':416,'bagging_freq':3,'reg_lambda':0.01,'num_leaves':20             
    }
matrix = lgbm.Dataset(treatment_x, label=treatment_y)
cv_result = lgbm.cv(params, matrix, num_boost_round=5000, nfold=5, stratified=True, 
                              early_stopping_rounds=50, seed=42, verbose_eval=50)

In [74]:
dataset

,age,stores_count,transaction_id,sum_regular_points_received,sum_express_points_received,sum_regular_points_spent,sum_express_points_spent,mean_regular_points_received,mean_express_points_received,mean_regular_points_spent,...,diff_mean_product_quantity_last30_store,mean_sums_per_store,diff_purchase_sums_total_store,diff_purchase_sums_last30_store,first_issue_unixtime,first_redeem_unixtime,first_redeem_delay,gender_M,gender_F,gender_U
client_id,,,,,,,,,,,,,,,,,,,,,
000012768d,45,3,4,392.7,0.0,0.0,0.0,7.551923,0.000000,0.000000,...,4.303419,277.829145,422.920855,333.170855,-1.639580,1.769781,3.409360,0,0,1
000036f903,72,5,32,348.3,360.0,0.0,0.0,2.150000,2.222222,0.000000,...,-0.235110,429.603072,-123.196822,-99.825294,-1.786588,-0.133929,1.652659,0,1,0
000048b7a6,68,2,8,218.8,0.0,0.0,0.0,3.907143,0.000000,0.000000,...,-4.475936,451.454465,20.045535,-102.454465,-0.270457,0.000000,0.270457,0,1,0
000073194a,60,1,17,518.8,0.0,-576.0,0.0,6.326829,0.000000,-7.024390,...,-5.017539,748.605569,-183.817334,-145.266998,-1.523791,-1.793599,-0.269808,0,1,0
00007c7133,67,2,11,459.7,0.0,-1680.0,0.0,5.538554,0.000000,-20.240964,...,-0.231788,344.526998,266.367548,35.473002,-1.639897,1.653310,3.293207,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
fffece623e,67,3,24,287.8,0.0,-544.0,0.0,2.459829,0.000000,-4.649573,...,-2.223408,314.573589,-42.489839,-131.009589,-0.596172,0.961591,1.557763,0,0,1
ffff3dfff8,56,1,17,1040.7,0.0,-728.0,0.0,10.840625,0.000000,-7.583333,...,2.025641,414.295112,13.048417,65.594888,-0.457501,-1.978659,-1.521159,0,1,0
ffffaab9da,23,1,7,394.9,0.0,-66.0,0.0,6.581667,0.000000,-1.100000,...,11.017218,291.860324,249.711105,266.139676,-0.588319,-0.832515,-0.244196,0,1,0
